# BERT - CRISPRoff


Tutorial adapted from https://colab.research.google.com/github/antonio-f/BERT_from_scratch/blob/main/BERT_from_scratch.ipynb#scrollTo=qwf2L-wZFJo0

### 0. Useful links

https://datascience.stackexchange.com/questions/54412/how-to-add-a-cnn-layer-on-top-of-bert

### 1. Packages and Data

In [8]:
import os
import json

n_sequences = 10
kmer_length = 5


In [9]:
import os
from subprocess import PIPE, run
import time
import pandas as pd
import numpy as np 
import pprint
import requests

pp = pprint.PrettyPrinter()

Data = pd.read_csv('/Users/sergiomares/Desktop/Nunez/Jin file/TSS_CpG_crispriphenotype_table.txt', delimiter = '\t',header = 0)

b = []

for i in range (n_sequences):
    datas = requests.get('http://togows.org/api/ucsc/hg19/'+ str(Data['chromosome'][i])+':'+str(int(Data["Primary TSS, 5'"][i]-1000))+'-'+str(Data["Primary TSS, 5'"][i]+1000)).text.replace('\n','')    
    b.append(datas)

Promoter_sequences = pd.DataFrame(b)

def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

Promoter_sequences = Promoter_sequences.apply(lambda x: getKmers(x[0], kmer_length), axis = 1)
Promoter_sequences = pd.DataFrame(Promoter_sequences)

Promoters_text = list(Promoter_sequences[0])

for item in range(len(Promoter_sequences)):
    Promoters_text[item] = ' '.join(Promoter_sequences[0][item])

y_data = Promoter_sequences.iloc[:, 0].values     

Promoter_sequences[1] = Data.index[:(len(Promoter_sequences))]
Promoter_sequences.columns = [['sequence', 'index']]
Promoter_sequences['text'] = pd.DataFrame(Promoters_text)

#dataset = dict(pd.MultiIndex.from_frame(Promoter_sequences[['index','text']]))

In [10]:
from datasets import Dataset

datasets = Dataset.from_pandas(Promoter_sequences[['index','text']])
datasets

Dataset({
    features: ["('index',)", "('text',)"],
    num_rows: 10
})

### 2. Building a tokenizer

In [11]:
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset:
      print(t, file=f)

dataset_to_text(datasets, "train.txt")

In [34]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files='train.txt', 
                vocab_size=30_522,
                min_frequency=6,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>']) 

In [36]:
import os 

os.mkdir('./liberto')
tokenizer.save_model('liberto')

['liberto/vocab.json', 'liberto/merges.txt']

### 3. Initializing the tokenizer

In [37]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('liberto',max_length = 3)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [39]:
tokens = tokenizer('gcgc')
tokens.input_ids

[0, 277, 277, 2]

### 4. Preparing the data

In [40]:
batch = tokenizer( Promoters_text , max_length=10, padding='max_length', truncation=True)

In [41]:
import torch

labels = torch.tensor([x for x in batch['input_ids']])
mask = torch.tensor([x for x in batch['attention_mask']])

In [43]:
labels

tensor([[   0,   88,  284,  261,  580,  584,  543,  861,  341,    2],
        [   0,  271,  305,  613,  541,  790,  414,  521,  432,    2],
        [   0,  262,  353,  431,  736,  761, 1220, 1110, 1217,    2],
        [   0,   88,   69,  312,  802, 1076,  866,  684,  512,    2],
        [   0,  284,  283,  422,  382,  392,  783,  492,  617,    2],
        [   0,  286,  271,  925, 1122,  511,  529,  526,  688,    2],
        [   0,   88,   71,  351,  427,  325,  288,  388, 1250,    2],
        [   0,  262,  285,  444,  494,  514,  493,  557,  426,    2],
        [   0,  287,  261, 1012,  681,  622,  517,  621, 1263,    2],
        [   0,  319,  262, 1286, 1030,  450,  892,  608,  537,    2]])

In [44]:
# make copy of labels tensor, this will be input_ids
input_ids = labels.detach().clone()
# create random array of floats with equal dims to input_ids
rand = torch.rand(input_ids.shape)
# mask random 15% where token is not 0 [PAD], 1 [CLS], or 2 [SEP]
# mask_arr = (rand < .15) * (input_ids != 0) * (input_ids != 1) * (input_ids != 2)
mask_arr = (rand < .15) * (input_ids > 2) 
# loop through each row in input_ids tensor (cannot do in parallel)
for i in range(input_ids.shape[0]):
    # get indices of mask positions from mask array
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    # mask input_ids
    input_ids[i, selection] = 3  # our custom [MASK] token == 3 

In [45]:
input_ids.shape

torch.Size([10, 10])

In [46]:
encodings = {'input_ids': input_ids, 'attention_mask': mask, 'labels': labels} 

In [47]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [48]:
dataset = Dataset(encodings) 

In [49]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) 

### 5. Training the model

#### A. Initializing the model

In [50]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=30_522,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
    ) 

In [51]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config) 

#### B. Training Preparation

In [52]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device) 

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [53]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

/Users/sergiomares/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### C. Training 

In [54]:
from tqdm import tqdm

epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item()) 

Epoch 1: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it, loss=8.85]


In [55]:
model.save_pretrained('./liberto')  # and don't forget to save liBERTo!

### 6. The real test

In [31]:
from transformers import pipeline

In [32]:
# Creates file too large for github to sync
#fill = pipeline('fill-mask', model='liberto', tokenizer='liberto')

In [56]:
fill(f'gcgc {fill.tokenizer.mask_token} tata ')

[{'score': 0.0016671427292749286,
  'token': 0,
  'token_str': '<s>',
  'sequence': 'gcgc tata '},
 {'score': 0.00134976115077734,
  'token': 2,
  'token_str': '</s>',
  'sequence': 'gcgc tata '},
 {'score': 0.00031376106198877096,
  'token': 262,
  'token_str': 'aa',
  'sequence': 'gcgcaa tata '},
 {'score': 0.00026194300153292716,
  'token': 15426,
  'token_str': '',
  'sequence': 'gcgc tata '},
 {'score': 0.00023920593957882375,
  'token': 11941,
  'token_str': '',
  'sequence': 'gcgc tata '}]

In [71]:
# To do: Fork ENFORMER from Deepmind repo and adapt it to include methylation information

In [61]:
import pysam
import pandas as pd

In [88]:
save = pysam.set_verbosity(0)
samfile = pysam.AlignmentFile("/Users/sergiomares/Downloads/293T1_sorted.bam", "rb")
a = 0
b = []


for read in samfile:
    if a != 50:
        print(read)
        a += 1



HS1:329:C3EGUACXX:5:2107:3073:68585	256	#0	10065	1	50M	*	0	0	CCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACACA	array('B', [34, 34, 34, 37, 37, 37, 37, 37, 39, 39, 38, 39, 39, 41, 41, 40, 41, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 40, 40, 40, 40, 41, 41, 41, 41, 41, 40, 41, 41, 41, 41, 41, 40, 40, 41, 41, 41, 41, 41])	[('AS', -12), ('XN', 0), ('XM', 2), ('XO', 0), ('XG', 0), ('NM', 2), ('MD', '47C1T0'), ('YT', 'UU'), ('NH', 4), ('CC', 'chr10'), ('CP', 135524461), ('HI', 0)]
HS1:329:C3EGUACXX:5:2312:18115:5456	272	#0	10532	0	50M	*	0	0	ACAGTACCACCGAAATCTGTGCAGAGGAGAACGCAGCTCCGCCCTCGCGG	array('B', [36, 35, 35, 35, 35, 33, 35, 33, 32, 35, 36, 37, 37, 37, 37, 37, 37, 39, 39, 37, 39, 39, 41, 41, 41, 41, 41, 41, 41, 41, 40, 41, 41, 41, 41, 41, 41, 39, 39, 39, 39, 39, 37, 37, 37, 37, 37, 34, 34, 34])	[('AS', -11), ('XN', 0), ('XM', 2), ('XO', 0), ('XG', 0), ('NM', 2), ('MD', '1G26C21'), ('YT', 'UU'), ('NH', 7), ('CC', 'chr12'), ('CP', 95042), ('HI', 0)]
HS1:329:C3EGUACXX:5:1212:12394:8998

In [25]:
len('CCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCAACACA')

counts = [34, 34, 34, 37, 37, 37, 37, 37, 39, 39, 38, 39, 39, 41, 41, 40, 41, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 40, 40, 40, 40, 41, 41, 41, 41, 41, 40, 41, 41, 41, 41, 41, 40, 40, 41, 41, 41, 41, 41]